# Welcome to JaxAHT!

*Please open this notebook in Colab.

In this tutorial, we will focus on introducing the core workflows for using the library.
We will:
- Demonstrate how to train teammates and ego agents, separately and as part of a single, unified workflow.
- Introduce the set of evaluation teammates and the evaluation framework
- Visualize learned policies

The project uses [Hydra](https://hydra.cc/) to manage algorithm and environment configurations, and [WandB](https://wandb.ai/) for logging.

Although the tutorial does not explicitly describe how to run the open-ended learning algorithms, these algorithm types may be run similarly to the MARL, teammate generation, and ego agent training algorithms.

Please see the project README for a full description of the project's design philosophy.
Our benchmark uses a multi-agent PPO implementation provided by [JaxMARL](https://github.com/FLAIROx/JaxMARL/tree/main).


# Install dependencies 📚

 ⚠️ Before beginning the tutorial, ensure you select a GPU or TPU from `Runtime > Change runtime type` ⚠️

 And Make sure that you select a runtime with python 3.11

 If none existant, you can use the following script to install py311

In [1]:
# !wget https://github.com/korakot/kora/releases/download/v0.11/py311.sh
# !bash ./py311.sh -b -f -p /usr/local
# !python -m ipykernel install --name "py311" --user

In [ ]:
%%shell
# clone repo and install packages
git clone https://github.com/carolinewang01/jax-aht.git
# if you need authentication, one way is to use
# git clone https://<GithubId>:<ghp_xxxxGithubTokenxxxx>@github.com/carolinewang01/jax-aht.git

cd jax-aht
pip install --upgrade pip
pip install -e .
# pip install numpy==1.25.* --upgrade # forcefully downgrade numpy; necessary for colab only

Use the following script to force (downgrade the versions)

You will be prompt to restart the session after numpy 1.25.2 is installed

In [3]:
%pip uninstall -y numpy
%pip install numpy==1.25.2 scipy==1.12.0
import numpy
print(numpy.__version__)

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
You can safely remove it manually.
You can safely remove it manually.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 27.2 MB/s  0:00:00m0:00:0100:01
Note: you may need to restart the kernel to use updated packages.
1.26.4


In [1]:
# Now check to make sure that we get 1.15.2
import numpy
print(numpy.__version__) # 1.25.2

1.25.2


In [ ]:
# change current working directory to rotate/ for the rest of this notebook
import os

path = os.getcwd()
if not path.endswith("jax-aht"):
    # if we are using the notebook from the repository directly
    if os.getcwd().endswith("jax-aht/docs"):
        os.chdir("..")
    # if we cloned the repository
    else:
        os.chdir("jax-aht")

print(os.getcwd()) # <path>/jax-aht

# verify that the jax installation can find the GPU/TPU
import jax
jax.devices()

/scratch/cluster/jyliu/Documents/jax-aht


[CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]

## Part 1.1 Training Teammates


In Jax-AHT, teammates may be trained using either MARL algorithms, or teammate generation algorithms. Each algorithm type has its own entry point, located at `marl/run.py` and `teammate_generation/run.py` respectively.
For this tutorial, we will train teammates on Level-Based Foraging (LBF) using IPPO.


**Viewing Metrics:**    We strongly recommend using WandB's UI to view the logged metrics. You will need a WandB account, and to set the logger to online mode. For the purposes of this tutorial, you can see a preview of the loggged metrics in the console. Note that the maximum return on LBF is 0.5.

In [9]:
%%bash
# train teammates on LBF using a MARL algorithm (IPPO w/parameter sharing)
PYTHONPATH=$(pwd) python marl/run.py task=lbf algorithm=ippo/lbf logger.mode=offline
# train using BRDiv instead. Note that we use the teammate_generation/ entry point instead of the marl/ entry point.
# PYTHONPATH=$(pwd) python teammate_generation/run.py task=lbf algorithm=brdiv/lbf logger.mode=offline

task:
  ENV_NAME: lbf
  ROLLOUT_LENGTH: 128
  ENV_KWARGS: {}
  TASK_NAME: lbf
algorithm:
  ALG: ippo
  TOTAL_TIMESTEPS: 500000.0
  ACTOR_TYPE: mlp
  NUM_CHECKPOINTS: 5
  GAMMA: 0.99
  GAE_LAMBDA: 0.95
  VF_COEF: 0.5
  MAX_GRAD_NORM: 1.0
  ANNEAL_LR: true
  SEED: 12345
  LR: 0.0001
  NUM_ENVS: 8
  UPDATE_EPOCHS: 15
  NUM_MINIBATCHES: 4
  CLIP_EPS: 0.03
  ENT_COEF: 0.01
  NUM_SEEDS: 3
  TRAIN_SEED: 20374
  ENV_NAME: lbf
  ENV_KWARGS: {}
  ROLLOUT_LENGTH: 128
ENV_NAME: lbf
ENV_KWARGS: {}
ROLLOUT_LENGTH: 128
TASK_NAME: lbf
label: default_label
name: lbf/ippo/default_label
logger:
  project: aht-benchmark
  entity: aht-project
  tags:
  - ippo
  - lbf
  - seed=20374
  - default_label
  mode: offline
  verbose: false
  log_train_out: true
  log_eval_out: true
local_logger:
  save_train_out: true
  save_eval_out: true



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.19.9
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: Adding directory to artifact (/scratch/cluster/jyliu/Documents/jax-aht/results/lbf/ippo/default_label/2025-09-08_00-21-09/saved_train_run)... Done. 0.2s
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: Train/returned_episode_returns ▁▁▁▁▁▂▂▃▂▃▄���▆▆▆▆▆▆▆▆▇▆▇▇████████████████
wandb:                     train_step ▁▁▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
wandb: 
wandb: Run summary:
wandb: Train/returned_episode_returns 0.47552
wandb:                     train_step 487
wandb: 
wandb: You can sync this run to

## Part 1.2: Training an Ego Agent Against Pretrained Teammate

Using the IPPO teammates trained in the last section, here, we will train a PPO ego agent to collaborate with those teammates, using the ego agent training entry point at `ego_agent_training/run.py`.

Please take a moment to look over the entry point code, to understand the overall pipeline.

In [10]:
! cat ego_agent_training/run.py

/scratch/cluster/jyliu/conda_envs/AHT/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


'''Main entry point for running ego agent training algorithms against a fixed partner population.'''
import hydra
from omegaconf import OmegaConf

from common.plot_utils import get_metric_names
from common.wandb_visualizations import Logger
from evaluation.heldout_eval import run_heldout_evaluation, log_heldout_metrics
from ppo_ego import run_ego_training
from ego_agent_training.ppo_br import run_br_training


@hydra.main(version_base=None, config_path="configs", config_name="base_config_ego")
def run_training(cfg):
    '''Runs the ego agent training against a fixed partner population.'''
    print(OmegaConf.to_yaml(cfg, resolve=True))
    wandb_logger = Logger(cfg)

    if cfg["algorithm"]["ALG"] == "ppo_ego":
        ego_params, ego_policy, init_ego_params = run_ego_training(cfg, wandb_logger)
    elif cfg["algorithm"]["ALG"] == "ppo_br":
        ego_params, ego_policy, init_ego_params = run_br_training(cfg, wandb_logger)

    if cfg["run_heldout_eval"]:
        metric_names = get_me

### 1.2.1: Setting the Partner Config

The first step to training an ego agent is to specify what teammates the ego agent should be trained with.
The ego agent training code looks for a partner config within the algorithm block of the master config file, located at `rotate/ego_agent_training/configs/base_config_ego.yaml`.



In [ ]:
! cat ego_agent_training/configs/base_config_ego.yaml

For the ppo_ego algorithm, a default partner config is specified within the algorithm-specific config directory, at `rotate/ego_agent_training/configs/algorithm/ppo_ego/_base_.yaml`. This default partner config is automatically imported and merged into the master config. The path (and other partner config values) should be set by the user, either directly within the config or via the command line. We will take the latter approach in this tutorial.


In [ ]:
! cat ego_agent_training/configs/algorithm/ppo_ego/_base_.yaml

In [11]:
# First, let's find the teammate checkpoint directory
import glob

checkpoint_base_dir = 'results/lbf/ippo/default_label/'
checkpoint_dir = sorted(glob.glob(os.path.join(checkpoint_base_dir, '*')))[-1]
checkpoint_dir = os.path.join(checkpoint_dir, "saved_train_run")
assert os.path.exists(checkpoint_dir), "Error: checkpoint directory not found."
print(f"Found checkpoint directory: {checkpoint_dir}")

Found checkpoint directory: results/lbf/ippo/default_label/2025-09-08_00-21-09/saved_train_run


### 1.2.2: Train the ego agent!

Now let's train the ego agent, directly specifying the partner path as a command line argument. We reduce the training time to 1 million steps, so that the ego agent trains in a couple minutes. We also turn off the evaluation against the heldout set, which is explained in the next section.

In [12]:
# Now let's train the ego agent, directly specifying the partner path from the command line
# total training timesteps is reduced to 1 million steps for this tutorial.
! PYTHONPATH=$(pwd) python ego_agent_training/run.py task=lbf algorithm=ppo_ego/lbf algorithm.partner_agent.path={checkpoint_dir} algorithm.TOTAL_TIMESTEPS=1e6 run_heldout_eval=false logger.mode=offline

task:
  ENV_NAME: lbf
  ROLLOUT_LENGTH: 128
  ENV_KWARGS: {}
  TASK_NAME: lbf
algorithm:
  ALG: ppo_ego
  EGO_ACTOR_TYPE: s5
  NUM_EGO_TRAIN_SEEDS: 3
  TOTAL_TIMESTEPS: 1000000.0
  NUM_CHECKPOINTS: 5
  NUM_ENVS: 8
  LR: 5.0e-05
  UPDATE_EPOCHS: 10
  NUM_MINIBATCHES: 4
  GAMMA: 0.99
  GAE_LAMBDA: 0.95
  CLIP_EPS: 0.1
  ENT_COEF: 0.0001
  VF_COEF: 0.5
  MAX_GRAD_NORM: 1.0
  ANNEAL_LR: false
  partner_agent:
    name: ippo
    path: results/lbf/ippo/default_label/2025-09-08_00-21-09/saved_train_run
    actor_type: mlp
    ckpt_key: final_params
    idx_list:
    - 0
  S5_D_MODEL: 128
  S5_SSM_SIZE: 128
  S5_ACTOR_CRITIC_HIDDEN_DIM: 1024
  FC_N_LAYERS: 3
  TRAIN_SEED: 12345
  NUM_EVAL_EPISODES: 20
  ENV_NAME: lbf
  ENV_KWARGS: {}
  ROLLOUT_LENGTH: 128
global_heldout_settings:
  EVAL_SEED: 34957
  MAX_EPISODE_STEPS: 128
  NUM_EVAL_EPISODES: 64
  AGGREGATE_STAT: mean
  NORMALIZE_RETURNS: true
heldout_set:
  lbf:
    ippo_mlp:
      path: eval_teammates/lbf/ippo/2025-04-21_23-41-17/saved_trai

# 2. Evaluation

In AHT research, ego agents are often evaluated based on the returns achieved during collaboration with a *heldout set* of evaluation agents---i.e., agents that should not have been seen during training.
The JaxAHT benchmark provides a heldout evaluation set for LBF, and the 5 classic Overcooked tasks.
The rliable library is used to compute bootstrapped metrics across the heldout set, by treating each heldout agent as a task.
In this section of the tutorial, we will introduce the heldout agent config, demonstrate how to download the agents, and visualize them.

*Note that the ego agent training entry point evaluate the ego agent against the heldout set by default. We turned this off in the last section, but you simply need to set the `run_heldout_eval` argument to true in order to enable evaluation.

 ## 2.1 Download the Heldout Set


 The heldout evaluation set consists of both agents trained via RL, and manually programmed heuristic agents. The heuristic agents are located under the `agents/lbf/` and `agents/overcooked/` directories, while the following code will download the RL agent checkpoints to an `eval_teammates/` directory.  



In [13]:
# first, let's download the RL agents in the heldout set
! python download_eval_data.py

Starting download & extraction: https://drive.google.com/file/d/1pS0wvJDzOZa954RADF_j9ETR74THzh8I/view?usp=sharing -> results/
Downloading...
From: https://drive.google.com/uc?id=1pS0wvJDzOZa954RADF_j9ETR74THzh8I
To: /scratch/cluster/jyliu/Documents/jax-aht/results/downloaded_gdrive_file.zip
100%|██████████████████████████████████████| 5.94k/5.94k [00:00<00:00, 26.8MB/s]
Downloaded results/downloaded_gdrive_file.zip (5942 bytes).
Unzipping results/downloaded_gdrive_file.zip to temporary directory /tmp/tmpxsm583sd...
Successfully unzipped results/downloaded_gdrive_file.zip to /tmp/tmpxsm583sd.
Processing and moving files from '/tmp/tmpxsm583sd/best_heldout_returns' to 'results/'...
Successfully moved 6 file(s) to results/.
Cleaning up temporary extraction directory: /tmp/tmpxsm583sd
Download completed successfully for best_returns_teammates.
Starting download & extraction: https://drive.google.com/file/d/1KjBV2GekKdRBiK6QSGe2vYx2ThXlG7X7/view?usp=sharing -> eval_teammates/
Downloading..

## 2.2. Heldout Agent Config

The evaluation teammate set is specified in the `global_heldout_settings.yaml` file, which is imported by other config files throughout the codebase as needed, to perform the heldout evaluation.

Notice that the teammate config format looks similar to the format used to specify the ego agent training teammates! This is because all teammate configs are parsed by functions within the `common/agent_loader_from_config.py` file. Please see the README for more details about the teammate config.

In [ ]:
! cat evaluation/configs/global_heldout_settings.yaml

## 2.3. Visualizing Agents

We provide functions to visualize agents at `evaluation/vis_episodes.py`.
Scripts to test the heuristic agents on LBF and Overcooked are also provided at `tests/test_lbf_agents.py` and `tests/test_overcooked_agents.py`.

In [14]:
import os
import numpy as np
from typing import Dict, Tuple

from IPython.display import HTML
import time
import jax

from envs import make_env
from agents.lbf import RandomAgent, SequentialFruitAgent

def run_episode(env, agent0, agent1, key) -> Tuple[Dict[str, float], int]:
    """Run a single episode with two heuristic agents.
    """
    # Reset environment
    key, subkey = jax.random.split(key)
    obs, state = env.reset(subkey)

    # Initialize episode tracking
    done = {agent: False for agent in env.agents}
    done['__all__'] = False
    total_rewards = {agent: 0.0 for agent in env.agents}
    num_steps = 0

    # Initialize agent states
    agent0_state = agent0.init_agent_state(0)
    agent1_state = agent1.init_agent_state(1)

    # Initialize state sequence
    state_seq = []
    while not done['__all__']:
        # Get actions from both agents with their states
        key, act0_rng, act1_rng = jax.random.split(key, 3)

        action0, agent0_state = agent0.get_action(obs["agent_0"], state, agent0_state, act0_rng)
        action1, agent1_state = agent1.get_action(obs["agent_1"], state, agent1_state, act1_rng)

        actions = {"agent_0": action0, "agent_1": action1}

        # Step environment
        key, subkey = jax.random.split(key)
        obs, state, rewards, done, info = env.step(subkey, state, actions)
        state_seq.append(state)

        # Update rewards
        for agent in env.agents:
            total_rewards[agent] += rewards[agent]

        num_steps += 1

    print(f"Episode finished. Total states collected: {len(state_seq)}")
    return total_rewards, num_steps, state_seq

def main(num_episodes,
         max_steps=100,
         visualize=False,
         save_video=False):
    print("Initializing environment...")
    env = make_env(env_name="lbf", env_kwargs={"time_limit": max_steps})

    print("Initializing agents...")
    # choices: lexicographic, reverse_lexicographic, column_major, reverse_column_major, nearest_agent, farthest_agent
    agent0 = SequentialFruitAgent(grid_size=7, num_fruits=3, ordering_strategy='lexicographic') # boxed
    agent1 = SequentialFruitAgent(grid_size=7, num_fruits=3, ordering_strategy='lexicographic') # not boxed

    print("Agent 0:", agent0.get_name())
    print("Agent 1:", agent1.get_name())

    # Run multiple episodes
    key = jax.random.PRNGKey(0)

    returns = []
    state_seq_all = []
    for episode in range(num_episodes):
        print(f"\nEpisode {episode + 1}/{num_episodes}")
        key, subkey = jax.random.split(key)
        total_rewards, num_steps, ep_states = run_episode(env, agent0, agent1, subkey)
        state_seq_all.extend(ep_states)  # Changed from += to extend for better list handling
        print(f"Total states in sequence after episode: {len(state_seq_all)}")

        # Calculate episode return
        episode_return = np.mean(list(total_rewards.values()))
        returns.append(episode_return)

        print(f"\nEpisode {episode + 1} finished:")
        print(f"Total steps: {num_steps}")
        print(f"Mean episode return: {episode_return:.2f}")
        print("Episode returns per agent:")
        for agent in env.agents:
            print(f" {agent}: {total_rewards[agent]:.2f}")

    # Print statistics
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    print(f"\nStatistics across {num_episodes} episodes:")
    print(f"Mean return: {mean_return:.2f} ± {std_return:.2f}")

    anim = env.animate(state_seq_all, interval=150)
    return anim

anim = main(num_episodes=5, max_steps=30)
HTML(anim.to_html5_video())

/scratch/cluster/jyliu/conda_envs/AHT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing environment...
Initializing agents...
Agent 0: SequentialFruitAgent
Agent 1: SequentialFruitAgent

Episode 1/5
Episode finished. Total states collected: 13
Total states in sequence after episode: 13

Episode 1 finished:
Total steps: 13
Mean episode return: 0.50
Episode returns per agent:
 agent_0: 0.50
 agent_1: 0.50

Episode 2/5
Episode finished. Total states collected: 16
Total states in sequence after episode: 29

Episode 2 finished:
Total steps: 16
Mean episode return: 0.50
Episode returns per agent:
 agent_0: 0.50
 agent_1: 0.50

Episode 3/5
Episode finished. Total states collected: 30
Total states in sequence after episode: 59

Episode 3 finished:
Total steps: 30
Mean episode return: 0.00
Episode returns per agent:
 agent_0: 0.00
 agent_1: 0.00

Episode 4/5
Episode finished. Total states collected: 14
Total states in sequence after episode: 73

Episode 4 finished:
Total steps: 14
Mean episode return: 0.50
Episode returns per agent:
 agent_0: 0.50
 agent_1: 0.50

Epis

<IPython.core.display.Javascript object>

RuntimeError: Requested MovieWriter (ffmpeg) not available